# nb

> Create FastHTML from a NB

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from pathlib import Path
import json, yaml
from execnb.nbio import *
from typing import Callable
from functools import partial

In [ ]:
#| export
hdrs = (MarkdownJS(), HighlightJS(langs=['python', 'javascript', 'html', 'css']), )

In [ ]:
if IN_NOTEBOOK:
    from fasthtml.jupyter import *
    app, rt = jupy_app(hdrs=hdrs)
    server = JupyUvi(app)

In [ ]:
example_nb_dir = Path('../example_nbs/')
with open(example_nb_dir/'explaining_xt_components.ipynb', 'r') as f: xt_nb = json.load(f)

In [ ]:
#| export
def strip_list(l, val='\n'):
    start, end = 0, len(l)
    while start < end and l[start] == val: start += 1
    while end > start and l[end - 1] == val: end -= 1
    return l[start:end]

# Render Cells

In [ ]:
#| export
def render_md(c,container=Div): return container(c,cls="marked")

In [ ]:
#| export
def render_md_cell(cell,render_md=render_md):
    assert cell['cell_type'] == 'markdown'
    return render_md(''.join(strip_list(cell['source'])))

In [ ]:
#| export
def get_nb_lang(nb): return nb['metadata']['kernelspec']['language']

In [ ]:
#| export
def render_code_source(cell,lang='python',render_md=render_md):
    if cell['source']==[]: return ''
    code = f'''```{lang}\n{''.join(strip_list(cell['source']))}'''
    return render_md(code)

In [ ]:
#| export
def render_code_output(cell,lang='python', render_md=render_md):
    res = []
    if len(cell['outputs'])==0: ''
    for output in cell['outputs']:
        if output['output_type'] == 'execute_result':
            data = output['data']
            if 'text/markdown' in data.keys(): 
                res.append(NotStr(''.join(strip_list(data['text/markdown'][1:-1]))))
            elif 'text/plain' in data.keys(): 
                res.append(''.join(strip_list(data['text/plain'])))
        if output['output_type'] == 'stream':
            res.append(''.join(strip_list(output['text'])))
    if res: return render_md(*res, container=Footer)

In [ ]:
nbs_dir = Path('../example_nbs/')
with open(nbs_dir/'00_core.ipynb', 'r') as f: nb = json.load(f)

# Frontmatter

In [ ]:
#| export
_RE_FM_BASE=r'''^---\s*
(.*?\S+.*?)
---\s*'''

In [ ]:
#| export
_re_fm_nb = re.compile(_RE_FM_BASE+'$', flags=re.DOTALL)
_re_fm_md = re.compile(_RE_FM_BASE, flags=re.DOTALL)

In [ ]:
#| export
def _fm2dict(s:str, nb=True):
    "Load YAML frontmatter into a `dict`"
    re_fm = _re_fm_nb if nb else _re_fm_md
    match = re_fm.search(s.strip())
    return yaml.safe_load(match.group(1)) if match else {}

In [ ]:
#| export
def _md2dict(s:str):
    "Convert H1 formatted markdown cell to frontmatter dict"
    if '#' not in s: return {}
    m = re.search(r'^#\s+(\S.*?)\s*$', s, flags=re.MULTILINE)
    if not m: return {}
    res = {'title': m.group(1)}
    m = re.search(r'^>\s+(\S.*?)\s*$', s, flags=re.MULTILINE)
    if m: res['description'] = m.group(1)
    r = re.findall(r'^-\s+(\S.*:.*\S)\s*$', s, flags=re.MULTILINE)
    if r:
        try: res.update(yaml.safe_load('\n'.join(r)))
        except Exception as e: warn(f'Failed to create YAML dict for:\n{r}\n\n{e}\n')
    return res

In [ ]:
#| export
def get_frontmatter(source,     # metatadata source (jupyter cell or md content)
                    nb_file=True,    # Is jupyter nb or qmd file
                    md_fm=False # md or raw style frontmatter
                   ):
    if not nb_file: return _fm2dict(source)
    if md_fm:       return _md2dict(source.source)
    return _fm2dict(source.source, nb_file)    

get_frontmatter_raw = partial(get_frontmatter, md_fm=False)
get_frontmatter_md =  partial(get_frontmatter, md_fm=True)

In [ ]:
nb = read_nb('../example_nbs/by_example.ipynb')
get_frontmatter(nb.cells[0], md_fm=True)

{'title': 'FastHTML By Example',
 'description': 'An introduction to FastHTML from the ground up, with four complete examples',
 'order': 2}

In [ ]:
get_frontmatter_md(read_nb('../example_nbs/by_example.ipynb').cells[0])

{'title': 'FastHTML By Example',
 'description': 'An introduction to FastHTML from the ground up, with four complete examples',
 'order': 2}

In [ ]:
#| export
def render_frontmatter(fm):
    return Div(cls='frontmatter')(H1(fm['title']),P(fm['description']))

# Render NB

In [ ]:
#| export
def render_nb(fpath, # Path to Jupyter Notebook
              wrapper=Main, #Wraps entire rendered NB, default is for pico
              cls='container', # cls to be passed to wrapper, default is for pico
              md_cell_wrapper=Div, # Wraps markdown cell
              md_fn=render_md_cell, # md cell -> rendered html
              code_cell_wrapper=Card, # Wraps Source Code (body) + Outputs (footer)
              cd_fn=render_code_source, # code cell -> code source rendered html
              out_fn=render_code_output, # code cell -> code output rendered html
              get_fm=get_frontmatter_md, # How to read frontmatter cell
              fm_fn:None|Callable=render_frontmatter, # Frontmatter -> FT components
              **kwargs # Passed to wrapper
             ): 
    nb = read_nb(fpath)
    fname = Path(fpath).name
    res, content_start_idx = [], 0
    if fm_fn: 
        content_start_idx = 1
        fm = get_fm(nb.cells[0])
        res.append(fm_fn(fm))
    for cell in nb.cells[content_start_idx:]:
        if cell['cell_type']=='code':
            s,o = cd_fn(cell), out_fn(cell)
            res.append(code_cell_wrapper(s,o))
        elif cell['cell_type']=='markdown': 
            res.append(md_cell_wrapper(md_fn(cell)))
    return wrapper(cls=cls)(*res)

In [ ]:
rendered_nb = render_nb('../example_nbs/by_example.ipynb',md_fm=True)

In [ ]:
xt_nb = read_nb('../example_nbs/by_example.ipynb')
get_frontmatter(xt_nb.cells[0], md_fm=True)

{'title': 'FastHTML By Example',
 'description': 'An introduction to FastHTML from the ground up, with four complete examples',
 'order': 2}

In [ ]:
# render_nb('../example_nbs/by_example.ipynb',md_fm=True)

In [ ]:
if IN_NOTEBOOK:
    @app.get
    def index():
        return render_nb('../example_nbs/by_example.ipynb',md_fm=True)

In [ ]:
# HTMX()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()